In [ ]:
# Instalamos Spark para Python
!pip install pyspark

import os

# Instalamos Java SDK 8
!apt-get install -y openjdk-8-jdk -qq > /dev/null
!echo $(/usr/libexec/java_home -v 1.8)

#set environment variable
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
!echo 2 | update-alternatives --config java

/bin/bash: line 1: /usr/libexec/java_home: No such file or directory

There are 2 choices for the alternative java (providing /usr/bin/java).

  Selection    Path                                            Priority   Status
------------------------------------------------------------
  0            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      auto mode
  1            /usr/lib/jvm/java-11-openjdk-amd64/bin/java      1111      manual mode
* 2            /usr/lib/jvm/java-8-openjdk-amd64/jre/bin/java   1081      manual mode

Press <enter> to keep the current choice[*], or type selection number: 

In [ ]:
from pyspark import SparkContext
sc = SparkContext("local", "Prueba")

root_path = '/content/'

ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=Prueba, master=local) created by __init__ at /tmp/ipython-input-995944444.py:2 

# Trabajo práctico 2

Análisis de uso de un videojuego donde jugadores se conectan a diario para combatir con otros. En un combate, un jugador “retador”
elije un oponente (jugador “retado”) y como resultado del combate el retador obtiene un puntaje y el tiempo en segundos que duró el combate. Por cada combate, los servidores del videojuego almacena la siguiente información:

• ID_Jugador_Retador

• ID_Jugador_Retado

• Puntaje obtenido (por el retador)

• Tiempo del combate en segundos

Todos los jugadores pueden participar cuántas veces deseen y por cada combate obtienen un puntaje y también se registra el tiempo que duró el combate. Un jugador puede combatir todas las veces que quiera.

Un fragmento del dataset de este jugador podría ser:
ID_Jug_Retador ID_Jugador_Retado Puntos Tiempo
231            492                1054   621
231            492                2068   504
231            98                 789    302
492             01                5462   955

Se quieren obtener algunas estadísticas para armar el ranking semanal. Al finalizar la semana se desea saber.
1) El jugador más “retador” y el jugador más “retado”.
2) El jugador que más puntos obtuvo en promedio :
𝑃𝑃𝑖 = (𝑝𝑢𝑛𝑡𝑎𝑗𝑒_𝑡𝑜𝑡𝑎𝑙_𝑑𝑒_𝑡𝑜𝑑𝑜𝑠_𝑙𝑜𝑠_𝑐𝑜𝑚𝑏𝑎𝑡𝑒𝑠𝑖 + 1) / (𝑐𝑎𝑛𝑡𝑖𝑑𝑎𝑑_𝑑𝑒_𝑐𝑜𝑚𝑏𝑎𝑡𝑒𝑠_𝑐𝑜𝑚𝑜_𝑟𝑒𝑡𝑎𝑑𝑜𝑟𝑖 + 1 )
(el +1 en el numerador y en el denominador es para evitar divisiones por cero en el caso que un jugador no haya “retado” a nadie, en cuyo caso tendrá el puntaje mínimo de 1).

3) Todos los jugadores que “retaron” a más de H oponentes distintos (H es un parámetro de la consulta).

4) El top 10 de los jugadores con mejor puntaje heroico. El puntaje PH de un jugador i se calcula como:
𝑃𝐻𝑖 = 𝛼 (∑𝑛 𝑗=1 𝑃𝐻𝑗. 𝑃𝑃𝑖/𝑃𝑃𝑗) + (1 − 𝛼)
donde n es la cantidad de jugadores “retados” por el jugador i, PHj es el puntaje P del j-ésimo jugador retado por i, PPi el puntaje promedio del jugador i, PPj el puntaje promedio del j-ésimo jugador retado por i y 𝛼 es un parámetro del algoritmo.
Cómo para calcular el puntaje PH de un jugador i previamente hay que calcular el puntaje de los jugadores a quiénes retó (y a su vez éstos pueden haber retado al jugador i), el cálculo se realiza de manera iterativa.
Se comienza asignando el mismo puntaje heroico con un valor arbitrario a todos los jugadores (tiempo 0), con esos puntajes arbitrarios se calcula el puntaje usando la fórmula detallada para obtener los nuevos puntajes (tiempo 1), con éstos se calculan los nuevos puntajes (tiempo 2) y así se continúa hasta alcanzar una convergencia en los valores. Al final de este documento hay un ejemplo que explica este cálculo iterativo.
Implemente la cantidad de jobs necesarios para resolver las cuatro consultas. Utilice el archivo jugadores.txt para realizar las pruebas (Se ofrecen los resultados de cada consulta).

In [ ]:
partidas = sc.textFile(root_path + "input.txt")

def parse_line(line):
    parts = line.strip().split("\t")
    retador = parts[0]
    retado = parts[1]
    puntos = int(parts[2])
    tiempo = int(parts[3])
    return (retador, retado, puntos, tiempo)

partidas = partidas.map(parse_line)

In [ ]:
retador_counts = partidas.map(lambda x: (x[0], 1)) \
                       .reduceByKey(lambda a, b: a + b)

retado_counts = partidas.map(lambda x: (x[1], 1)) \
                      .reduceByKey(lambda a, b: a + b)

mas_retador = retador_counts.max(key=lambda x: x[1])

mas_retado = retado_counts.max(key=lambda x: x[1])

print(f"Jugador más retador: {mas_retador[0]} con {mas_retador[1]} combates")
print(f"Jugador más retado: {mas_retado[0]} con {mas_retado[1]} combates")

Jugador más retador: 1 con 18 combates
Jugador más retado: 43 con 15 combates


In [ ]:
puntos_y_cant = partidas.map(lambda x: (x[0], (x[2], 1))) \
                      .reduceByKey(lambda a, b: (a[0]+b[0], a[1]+b[1]))

pp_por_jugador = puntos_y_cant.mapValues(lambda v: (v[0]+1)/(v[1]+1))

jugador_max_pp = pp_por_jugador.reduce(lambda a, b: a if a[1] >= b[1] else b)

print(f"Jugador con mayor PP: {jugador_max_pp[0]} con promedio {jugador_max_pp[1]:.2f}")

Jugador con mayor PP: 53 con promedio 3504.43


In [ ]:
H = int(input("Ingrese el número mínimo de oponentes a buscar: "))

retador_oponente = partidas.map(lambda x: (x[0], x[1]))

oponentes_distintos = retador_oponente.distinct() \
                                      .groupByKey() \
                                      .mapValues(lambda vals: len(set(vals)))

jugadores_filtrados = oponentes_distintos.filter(lambda x: x[1] > H)

resultado = jugadores_filtrados.collect()
for jugador, cant in resultado:
    print(f"Jugador {jugador} retó a {cant}")


Ingrese el número mínimo de oponentes a buscar: 11
Jugador 65 retó a 13
Jugador 55 retó a 12
Jugador 48 retó a 12
Jugador 1 retó a 15


In [ ]:

#Calcular puntajes promedio PP_i

puntos_y_cant = partidas.map(lambda x: (x[0], (x[2], 1))) \
                        .reduceByKey(lambda a, b: (a[0] + b[0], a[1] + b[1]))

pp_por_jugador = puntos_y_cant.mapValues(lambda v: (v[0] + 1) / (v[1] + 1)).cache()

#Incluir jugadores que solo fueron retados (no retadores)
todos_jugadores = partidas.flatMap(lambda x: [x[0], x[1]]).distinct()
pp_por_jugador = todos_jugadores.map(lambda j: (j, (0, 0))) \
    .leftOuterJoin(puntos_y_cant) \
    .mapValues(lambda v: v[1] if v[1] is not None else v[0]) \
    .mapValues(lambda v: (v[0] + 1) / (v[1] + 1)) \
    .cache()

#Inicializar PH_i
ph = todos_jugadores.map(lambda j: (j, 1.0))

#Construir grafo retador -> retados
edges = partidas.map(lambda x: (x[0], x[1])) \
                .distinct() \
                .groupByKey() \
                .mapValues(list) \
                .cache()

#Iteraciones para cálculo de PH
alpha = float(input("Ingrese el valor de α (ej. 0.85): "))
max_iter = 20
tol = 1e-6

def calcular_ph(i, oponentes, ph_dict, pp_dict, alpha):
    if not oponentes:
        return (i, (1 - alpha))
    suma = 0.0
    for op in oponentes:
        pp_i = pp_dict.get(i, 1.0)
        pp_j = pp_dict.get(op, 1.0)
        ph_j = ph_dict.get(op, 1.0)
        suma += ph_j * (pp_i / pp_j)
    return (i, alpha * suma + (1 - alpha))

for it in range(max_iter):
    # Broadcast de PH y PP para esta iteración
    ph_dict = dict(ph.collect())
    pp_dict = dict(pp_por_jugador.collect())
    bc_ph = sc.broadcast(ph_dict)
    bc_pp = sc.broadcast(pp_dict)

    # Calcular nuevo PH
    new_ph = edges.map(lambda x: calcular_ph(x[0], x[1], bc_ph.value, bc_pp.value, alpha))

    # Unir con jugadores que no tienen oponentes
    ph = todos_jugadores.map(lambda j: (j, 0.0)) \
                        .leftOuterJoin(new_ph) \
                        .mapValues(lambda v: v[1] if v[1] is not None else (1 - alpha))

    # Calcular diferencia total para convergencia
    diff = ph.join(sc.parallelize(list(ph_dict.items()))) \
             .mapValues(lambda x: abs(x[0]-x[1])) \
             .values().sum()

    print(f"Iteración {it+1}: diferencia total = {diff}")

    if diff < tol:
        print(f"Convergencia alcanzada en iteración {it+1}")
        break

#Top 10 jugadores por PH

top10 = ph.takeOrdered(10, key=lambda x: -x[1])
print("\nTop 10 jugadores por Puntaje Heroico (PH):")
for j, ph_val in top10:
    print(f"Jugador {j} -> PH = {ph_val:.4f}")


Ingrese el valor de α (ej. 0.85): .1
Iteración 1: diferencia total = 5238.341959690744
Iteración 2: diferencia total = 3115.893026327906
Iteración 3: diferencia total = 1664.9693644241734
Iteración 4: diferencia total = 1012.068401674345
Iteración 5: diferencia total = 596.4657715884292
Iteración 6: diferencia total = 356.4426736020848
Iteración 7: diferencia total = 212.63781709199054
